# CSE 153 Assignment 2

## Task 2: Harmonization

For this task, we will create a model that can generate harmonies and a backing track when provided a melody. (PROVIDE MORE DETAILS)

In [1]:
# IMPORTS
import pickle
import os
import sys
from processor import MidiEventProcessor
import pretty_midi as pyd
import numpy as np

We will use the [POP909 dataset](https://arxiv.org/abs/2008.07142), which contains 909 piano arrangements of popular songs. Each song is broken into 3 midi files (MELODY, BRIDGE (for the harmonies), and PIANO (for the accompaniment)). The dataset comes with information about beat, chords, and key changes. We will be using their provided data preprocessing scripts to access these midi files.

In [8]:
# Pre-processing script by Ziyu Wang, Ke Chen, Junyan Jiang, Yiyi Zhang, Maoran Xu, 
# Shuqi Dai, Guxian Bin, and Gus Xia (POP909 team)

total = 0
def preprocess_midi(path):
    global total
    data = pyd.PrettyMIDI(path)
    main_notes = []
    acc_notes = []
    for ins in data.instruments:
        acc_notes.extend(ins.notes)
    
    for i in range(len(main_notes)):
        main_notes[i].start = round(main_notes[i].start,2)
        main_notes[i].end = round(main_notes[i].end,2)
        
    for i in range(len(acc_notes)):
        acc_notes[i].start = round(acc_notes[i].start,2)
        acc_notes[i].end = round(acc_notes[i].end,2)
    
    main_notes.sort(key = lambda x:x.start)
    acc_notes.sort(key = lambda x:x.start)
    mpr = MidiEventProcessor()
    repr_seq = mpr.encode([main_notes, acc_notes])
    total += len(repr_seq)
    return repr_seq

def preprocess_pop909(midi_root, save_dir):
    save_py = []
    midi_paths = [f"{d}/{d}.mid" for d in os.listdir(midi_root)]
    i = 0
    
    for path in midi_paths:
        print(' ', end='[{}]'.format(path), flush=True)
        filename = midi_root + path
        try:
            data = preprocess_midi(filename)
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        save_py.append(data)
    save_py = np.array(save_py)
    print(save_py.size)
    np.save("pop909-event-token.npy", save_py)
            
    
# replace the folder with your POP909 data folder
preprocess_pop909("POP909/","midi_data/")

 [038/038.mid]

AttributeError: 'list' object has no attribute 'start'

@inproceedings{pop909-ismir2020,
    author = {Ziyu Wang* and Ke Chen* and Junyan Jiang and Yiyi Zhang and Maoran Xu and Shuqi Dai and Guxian Bin and Gus Xia},
    title = {POP909: A Pop-song Dataset for Music Arrangement Generation},
    booktitle = {Proceedings of 21st International Conference on Music Information Retrieval, {ISMIR}},
    year = {2020}
}